In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
import torch
print(f"현재 적용된 Torch 버전: {torch.__version__}")

In [ ]:
# necessary libraries
import torch
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from transformers import AutoImageProcessor, YolosForObjectDetection
import os
import glob

# --- 1. Model Loading ---
print("Loading the model...")
# Load the processor and model from Hugging Face
processor = AutoImageProcessor.from_pretrained("valentinafeve/yolos-fashionpedia")
model = YolosForObjectDetection.from_pretrained("valentinafeve/yolos-fashionpedia")
print("Model loaded successfully!")

# --- 2. Set up data path and create image list ---
# Set the path to your image folder
image_folder = r"D:/train" 
# List of image extensions to search for
image_extensions = ["*.jpg", "*.jpeg", "*.png"] 
image_paths = []
# Find all images with the specified extensions in the folder
for ext in image_extensions:
    image_paths.extend(glob.glob(os.path.join(image_folder, ext)))

if not image_paths:
    print(f"Warning: No images found in '{image_folder}'. Please check the path.")
else:
    print(f"Found {len(image_paths)} images to analyze.")


# --- 3. Helper Functions ---

def get_accessory_ids(model):
    """Gets the IDs for accessory names from the model's configuration."""
    accessory_names = [
        'hat', 'sunglasses', 'scarf', 'necktie', 'belt', 'earrings', 
        'necklace', 'watch', 'bracelet', 'bag', 'wallet', 'gloves', 'headband'
    ]
    label2id = model.config.label2id
    accessory_ids = {label2id[name] for name in accessory_names if name in label2id}
    return accessory_ids

def classify_style(result, accessory_ids):
    """Counts accessories to classify the style."""
    acc_count = sum(1 for label_id in result["labels"] if label_id.item() in accessory_ids)
    return "Maximal" if acc_count >= 2 else "Minimal"

def visualize_detection_and_style(image, results, style):
    """Draws detected objects and the final style result on the image."""
    plt.figure(figsize=(15, 15))
    ax = plt.gca()
    ax.imshow(image)

    # Display the final style result text at the top of the image
    ax.text(20, 60, f"Style: {style}", fontsize=40, color='white', 
            bbox=dict(facecolor='magenta', alpha=0.8))

    # Draw the detected objects
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = [round(i, 2) for i in box.tolist()]
        x, y, w, h = box[0], box[1], box[2] - box[0], box[3] - box[1]
        
        rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='cyan', facecolor='none')
        ax.add_patch(rect)
        
        label_text = f"{model.config.id2label[label.item()]}: {score:.2f}"
        ax.text(x, y - 10, label_text, fontsize=12, color='black', bbox=dict(facecolor='cyan', alpha=0.5))

    plt.axis("off")
    plt.show()

# --- 4. Main Pipeline ---

# Iterate through each found image for analysis.
for image_path in image_paths[:10]:
    try:
        print(f"\n{'='*25}\n[Starting Analysis]: {os.path.basename(image_path)}\n{'='*25}")
        
        # Open the image
        image_to_analyze = Image.open(image_path).convert("RGB")

        # 1. Prepare the image for the YOLOS model
        inputs = processor(images=image_to_analyze, return_tensors="pt")
        
        # 2. Run the model (inference)
        print("Running model inference...")
        outputs = model(**inputs)
        print("Inference complete!")
        
        # 3. Post-process the model output
        target_sizes = torch.tensor([image_to_analyze.size[::-1]])
        results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.8)[0]
        
        # 4. Classify the style
        accessory_ids = get_accessory_ids(model)
        predicted_style = classify_style(results, accessory_ids)
        
        # 5. Print the final results
        detected_items = [model.config.id2label[label.item()] for label in results["labels"]]
        print(f"\n--- Analysis Result ---")
        print(f"Detected items: {detected_items if detected_items else 'None'}")
        print(f"🧥 Final Style Classification: {predicted_style}")
        print("-----------------")
        
        # 6. Visualize the results
        visualize_detection_and_style(image_to_analyze, results, predicted_style)

    except Exception as e:
        print(f"Error: An error occurred while processing {os.path.basename(image_path)}: {e}")

In [ ]:
# 클래스 라벨 출력
labels = model.config.id2label

# 전체 라벨 출력
for idx, name in labels.items():
    print(f"{idx}: {name}")